In [ ]:
import logging
import os
import re

import pandas as pd

from pathlib import Path

%load_ext autoreload
%autoreload 2

# Disable logging of the program in the notebook
os.environ["LOGLEVEL"] = "CRITICAL"

logging.basicConfig()
logging.root.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)

logger = logging.getLogger('__main__')
logger.setLevel(os.environ.get("LOGLEVEL", logging.FATAL))

pybooklogger = logging.getLogger('pybook')
pybooklogger.setLevel(logging.DEBUG)

%aimport setup

from setup import *
from tad_help import TadRunner

# Track Layout
Calculate the layout of the dutch railway system


In [ ]:
layout_file =   "../data/prorail/parsed/netherlands-schiphol.json"
layout = Layout(layout_file)

In [ ]:
scenario_files = Path("../data/prorail/scenarios/TAD/Zwolle")
save_dir = r"C:\Users\erick\Documents\uni\Thesis\delay-replannning\experiments\results\tad"

In [ ]:
path_data = []
time_data = []
comp_data = []
# for scenario_file in os.listdir(scenario_files):
for scenario_file in ["2025-07-21_1.json"]:
    try:
        tad_exp = TadRunner(layout, scenario_files / scenario_file, save_dir)
        experiments = tad_exp.run("600", "o", "AMF", "SWK", default_direction=1)
        experiments[0].metadata = {'color': 'Red',   'label': '@SIPP',    'offset': 0, 'linestyle': 3}
        experiments[1].metadata = {'color': 'Blue',  'label': 'FlexSIPP', 'offset': 0, 'linestyle': 3}

        tad_exp.plot(experiments, save="600o-ZL", y_offset=-1200, include_expected_arrival=False)
        tad_exp.plot([experiments[0]], save="600o-ZL/atsipp", y_offset=-1200, include_expected_arrival=False)
        tad_exp.plot([experiments[1]], save="600o-ZL/flexsipp", y_offset=-1200, include_expected_arrival=False)

        path_data.extend(get_path_data(experiments, tad_exp.agent_df, scenario=scenario_file))
        time_data.extend([exp.get_running_time() | exp.get_label() | {'scenario': scenario_file} for exp in experiments])
        comp_data.extend([exp.get_complexity()   | exp.get_label() | {'scenario': scenario_file} for exp in experiments])
    except Exception as e:
        print(e)

# path_df = pd.DataFrame(path_data)
# path_df.to_csv(save_dir + "/600o-Zl.csv")
#
# time_df = pd.DataFrame(time_data)
# time_df.to_csv(save_dir + "/600o-Zl-time.csv")
#
# comp_df = pd.DataFrame(comp_data)
# comp_df.to_csv(save_dir + "/600o-Zl-comp.csv")

In [ ]:


tad_exp.plot(experiments, save="600o-ZL", y_offset=-1200, include_expected_arrival=False)
tad_exp.plot([experiments[0]], save="600o-ZL/atsipp", y_offset=-1200, include_expected_arrival=False)
tad_exp.plot([experiments[1]], save="600o-ZL/flexsipp", y_offset=-1200, include_expected_arrival=False)

In [ ]:
path_df = pd.read_csv(save_dir + "/600o-Zl.csv", index_col=0)
path_df

In [ ]:
def extract_tipping_point(df):
    def apply_func(df):
        result=df.groupby("delay_location").agg({"beta": lambda x: str(timedelta(seconds=x.max()))}).rename(columns={
            "beta": "tipping_point"})
        return result

    df["delay_location"] = df["delay_location"].str.split("|").apply(lambda x: x[0])
    df = df.groupby(by='trainNumber').apply(apply_func, include_groups=False)
    return df

path_df.groupby(["scenario", "label"]).apply(extract_tipping_point, include_groups=False)

In [ ]:
allowed_delay = 180

def td_str(td):
    return ':'.join(re.split(r'[:.]+', str(td)) [1:3])

def extract_tipping_point(df):
    def apply_func(df):

        result=df.groupby("Delay Location").agg({
            "alpha": "min",
            "beta": "max",
            "Delay Amount": "max",
        })
        # result = result.loc[result['beta'] < 900]
        result["Tipping Point a"] = result["alpha"].apply(lambda x: td_str(timedelta(seconds=x)))
        result["Tipping Point b"] = result["beta"].apply(lambda x: td_str(timedelta(seconds=x)))

        def tp_finder(x):
            new_tp = x['beta'] - max(0, x['Delay Amount'] - allowed_delay)
            if new_tp > 0:
                return td_str(timedelta(seconds=new_tp))
            return "-"

        # result[f"Tipping Point ({allowed_delay}s)"] = result.apply(tp_finder, axis=1)
        return result.sort_values("Tipping Point a", ascending=True).drop(columns=["alpha", "beta", "Delay Amount"])

    df["Delay Location"] = df["Delay Location"].str.split("|").apply(lambda x: x[0])
    df = df.groupby(by='Train').apply(apply_func, include_groups=False)
    return df

tp_df = path_df.rename(columns={
    "delay_amount": "Delay Amount",
    "delay_location": "Delay Location",
    "trainNumber": "Train",
    "scenario": "Scenario",
    "label": "Label",
})
tp_df["Scenario"] = tp_df["Scenario"].apply(lambda x: x.split(".")[0])
tp_df["Delay Location"] = tp_df["Delay Location"].apply(lambda x: x.split("-")[1])
tp_df = tp_df.groupby(["Scenario"]).apply(extract_tipping_point, include_groups=False)
# tp_df = tp_df.loc[tp_df.index.get_level_values('Delay Location').astype(str).str.contains("Zl")]
tp_df.to_latex(save_dir + "/600o-Zl.tex", escape=True)
tp_df

In [ ]:

tad_exp.plot(experiments, save="600o-ZL", y_offset=-1200, include_expected_arrival=False)

# Experiment Runtime
Take a route of an agent with many stops, and run from start to every stop as an experiment

## Results


### ATF Plot

### Blocking staircase diagram
Showing the route of the agent with the most stops, its quite long.


In [ ]:
tad_exp = TadRunner(layout, scenario_files / "2025-07-21_1.json", save_dir)
experiments = tad_exp.run("600", "o", "ZL", "MP", default_direction=1)

In [ ]:
kwargs = {'min_y': 0*60, 'max_y': (90)*60}
for exp in experiments:
    exp.s.plot(exp.agent.id, exp.block_intervals, exp.buffer_times, exp.recovery_times, True, start="ZL|", end="MP|", savefig=f"{save_dir}/2025-07-21_1/blocking_staircase_{exp.metadata['label'].replace(' ', '_')}.png", **kwargs)